# Final Project

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import randint as sp_randint
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor,BaggingRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score

plt.style.use('seaborn') #set same style for all plots
pd.options.mode.chained_assignment = None

output = pd.DataFrame(index=None, columns=['model','train_r2_score','test_r2_score'])

print("Libraries successfulling imported")

Libraries successfulling imported


Import data

In [10]:
data = pd.read_csv('ENB2012_data.csv')
data.head()


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [11]:
data.columns = ['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height',
                'Orientation', 'Glazing_Area', 'Glazing_Area_Distribution', 'Heating_Load', 'Cooling_Load']


Splitting the data

In [12]:
X = data[['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height',
                'Orientation', 'Glazing_Area', 'Glazing_Area_Distribution']]
Y = data[[ 'Heating_Load', 'Cooling_Load']]
Y1=data[['Heating_Load']]
Y2=data[['Cooling_Load']]
print(X.isnull().sum())

Relative_Compactness         0
Surface_Area                 0
Wall_Area                    0
Roof_Area                    0
Overall_Height               0
Orientation                  0
Glazing_Area                 0
Glazing_Area_Distribution    0
dtype: int64


Dividing the observations

In [13]:
Z = data[['Heating_Load','Cooling_Load']]

# The total load is the sum of the individual heating and cooling load
Z['Overall_load'] = Z['Heating_Load'] + Z['Cooling_Load']
Z['class'] = 1

Z.loc[Z.Overall_load < 42,['class']] = 0
Z.loc[Z.Overall_load > 70 , ['class']] = 2

yclass = Z['class']

xTraining, xTesting, yTraining, yTesting = train_test_split(X, yclass, random_state = 5)


Using minmax scaler

In [14]:
minMax = MinMaxScaler(feature_range= (0,1))
xTraining = minMax.fit_transform(xTraining)
xTesting = minMax.transform(xTesting)


Dividing the train and test data

In [15]:
#Test-Train split of data for regression models
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 5)

X_train_div, X_test_div, y1_train, y1_test = train_test_split(X, Y1, random_state = 5)

X_train_div, X_test_div, y2_train, y2_test = train_test_split(X, Y2, random_state = 5)


Using the minmax scaler on the dataset

In [16]:
minMax = MinMaxScaler(feature_range= (0,1))
X_train = minMax.fit_transform(X_train)
X_test = minMax.transform(X_test)

MODELING
Linear Regression

In [17]:
linear = LinearRegression()
multiOutputLinearReg = MultiOutputRegressor(linear,n_jobs=-1)
multiOutputLinearReg.fit(X_train,y_train)
train_r2_score=r2_score(y_train,multiOutputLinearReg.predict(X_train))
test_r2_score=r2_score(y_test,multiOutputLinearReg.predict(X_test))
output = output.append(pd.Series({'model':'Linear Regressor', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output

,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902539,0.899354


Linear SVM

In [18]:
param_grid = {"C": [1e0, 1e1, 1e2, 1e3, 1e4],"gamma": np.logspace(-2, 1, 2, 3, 5)}
grid_search_svm = MultiOutputRegressor(GridSearchCV(SVR(kernel='linear'), param_grid, cv=10,return_train_score=True))

grid_search_svm.fit(X_train, y_train)
train_r2_score=r2_score(y_train,grid_search_svm.predict(X_train))
test_r2_score=r2_score(y_test,grid_search_svm.predict(X_test))
output = output.append(pd.Series({'model':'SVM Linear', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output


,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902539,0.899354
1,SVM Linear,0.897451,0.896480


Ada Boosting on Linear SVM regressor

In [19]:
param =  { "n_estimators": [100,500,1000] }


base_svr=SVR(kernel='linear')
ada_svr = AdaBoostRegressor(base_estimator=base_svr,learning_rate = 0.7,random_state=10)
adaboost_svr = MultiOutputRegressor(GridSearchCV(ada_svr,param_grid=param,n_jobs=-1),n_jobs=-1)
adaboost_svr.fit(X_train, y_train)
train_r2_score=r2_score(y_train,adaboost_svr.predict(X_train))
test_r2_score=r2_score(y_test,adaboost_svr.predict(X_test))
output = output.append(pd.Series({'model':'Adaboost_LinearSVM', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output


,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902539,0.899354
1,SVM Linear,0.897451,0.896480
2,Adaboost_LinearSVM,0.896437,0.893522


Gradient Boosting regressor

In [20]:
param = {"min_samples_split": [3,4,5],
         "max_features": [2,3,4,5,6],
         'n_estimators': [100,500,1000]}

gradient = GradientBoostingRegressor(learning_rate =1.1,random_state=10)
gradient_mr = MultiOutputRegressor(GridSearchCV(gradient,param_grid=param,n_jobs=-1),n_jobs=-1)
gradient_mr.fit(X_train, y_train)
train_r2_score=r2_score(y_train,gradient_mr.predict(X_train))
test_r2_score=r2_score(y_test,gradient_mr.predict(X_test))
output = output.append(pd.Series({'model':'Gradient Boosting Regressor', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output


,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902539,0.899354
1,SVM Linear,0.897451,0.896480
2,Adaboost_LinearSVM,0.896437,0.893522
3,Gradient Boosting Regressor,0.999599,0.996793


4. Classification modeling
After running multioutput regressors we will now be running classification models

In [21]:
#Logistic Regression

param_grid = {'C':[0.01, 0.1, 0.5, 1, 2, 5, 10, 15, 20]}

model = LogisticRegression()

grid_search = GridSearchCV(model, param_grid, cv=10)
grid_search.fit(xTraining, yTraining)

print('Best parameters for efficiency classification {}'.format(grid_search.best_params_))

print('The Train Accuracy score for Logistic Reression is',accuracy_score(yTraining, grid_search.predict(xTraining)))
print('The Test Accuracy score for Logistic Reression is',accuracy_score(yTesting, grid_search.predict(xTesting)))


C:\Users\SahithiKaringula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SahithiKaringula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://sci

C:\Users\SahithiKaringula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SahithiKaringula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://sci

Best parameters for efficiency classification {'C': 0.5}
The Train Accuracy score for Logistic Reression is 0.8836805555555556
The Test Accuracy score for Logistic Reression is 0.8854166666666666


In [22]:
#Linear SVM

param_grid = {'C':[0.001, 0.01, 0.1, 0.5, 1, 10]}

model = LinearSVC()

model = LogisticRegression()

grid_search_Log_Class = GridSearchCV(model, param_grid, cv=5)
grid_search_Log_Class.fit(xTraining, yTraining)

print('Best parameters for efficiency classification {}'.format(grid_search_Log_Class.best_params_))

print('The Train Accuracy score for Linear SVC is',accuracy_score(yTraining, grid_search_Log_Class.predict(xTraining)))
print('The Test Accuracy score for Linear SVC is',accuracy_score(yTesting, grid_search_Log_Class.predict(xTesting)))


Best parameters for efficiency classification {'C': 0.5}
The Train Accuracy score for Linear SVC is 0.8836805555555556
The Test Accuracy score for Linear SVC is 0.8854166666666666


C:\Users\SahithiKaringula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SahithiKaringula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://sci

In [23]:
#gradient boosting regression

param_grid = {'max_features':['auto', 'log2'], 'max_depth':[5,10,15,20,50,60], 'learning_rate':[0.01,0.1,0.7,1]}

model =  GradientBoostingClassifier(random_state = 5, n_estimators = 500)


grid_search_GB_Class = GridSearchCV(model, param_grid, cv=5)
grid_search_GB_Class.fit(xTraining, yTraining)

print('Best parameters for efficiency classification {}'.format(grid_search_GB_Class.best_params_))

print('The Train Accuracy score for Gradient Boosting Classifier is',accuracy_score(yTraining, grid_search_GB_Class.predict(xTraining)))
print('The Test Accuracy score for Gradient Boosting Classifier is',accuracy_score(yTesting, grid_search_GB_Class.predict(xTesting)))


Best parameters for efficiency classification {'learning_rate': 0.01, 'max_depth': 5, 'max_features': 'log2'}
The Train Accuracy score for Gradient Boosting Classifier is 1.0
The Test Accuracy score for Gradient Boosting Classifier is 0.9895833333333334


 Neural Network Regression model

In [24]:
np.random.seed(5)

In [25]:

def create_model_regressor_heating():
    #create model
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer = 'normal', activation = 'relu')) # hidden layer
    model.add(Dense(10, kernel_initializer = 'normal', activation = 'linear'))
    model.add(Dense(1, kernel_initializer='normal'))
    #compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model


In [30]:

def create_model_regressor_cooling():
    #create model
    model2 = Sequential()
    model2.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model2.add(Dense(4, kernel_initializer = 'normal', activation = 'relu')) # hidden layer
    model2.add(Dense(10, kernel_initializer = 'normal', activation = 'linear'))
    model2.add(Dense(1, kernel_initializer='normal'))
    #compile model
    model2.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    #return model
    return model2


In [31]:
#Model training

param_grid = {'epochs':[50, 100, 200] , 'batch_size':[20, 50, 100]}


In [32]:
model1 = KerasRegressor(build_fn = create_model_regressor_cooling , verbose = 0)

grid_search_Keras_Reg = GridSearchCV(model1 , param_grid , cv =5)

grid_search_Keras_Reg.fit(X_train_div, y1_train)

print('Best parameters for cooling load {}'.format(grid_search_Keras_Reg.best_params_))

print('The Train R2 score is',r2_score(y1_train, grid_search_Keras_Reg.predict(X_train_div)))
print('The Test R2 score is',r2_score(y1_test, grid_search_Keras_Reg.predict(X_test_div)))


Best parameters for cooling load {'batch_size': 20, 'epochs': 100}
The Train R2 score is 0.7971272608990809
The Test R2 score is 0.7385246026418604


In [33]:
model2 = KerasRegressor(build_fn = create_model_regressor_heating , verbose = 0)
grid_search_Keras2 = GridSearchCV(model2 , param_grid , cv =10)

grid_search_Keras2.fit(X_train_div, y2_train)
print('Best parameters for heating load {}'.format(grid_search_Keras2.best_params_))

print('The Train R2 score is',r2_score(y2_train, grid_search_Keras2.predict(X_train_div)))
print('The Test R2 score is',r2_score(y2_test, grid_search_Keras2.predict(X_test_div)))


Best parameters for heating load {'batch_size': 100, 'epochs': 200}
The Train R2 score is 0.7130883635331999
The Test R2 score is 0.6734379399186448


5. Neural Network Classification model

In [39]:

# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(yTraining)
y_test = np_utils.to_categorical(yTesting)



Creating model function for NN

In [40]:

def model_classifier():
    #create model
    model = Sequential()
    model.add(Dense(8, input_dim=8, activation='relu'))
    #Layer 2 - Hidden layer
    model.add(Dense(4, activation='relu')) 
    #Output layer
    model.add(Dense(3,activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



Grid Search for NN

In [41]:
param_grid = {'epochs':[50, 200] , 'batch_size':[10, 50, 100]}

model = KerasClassifier(build_fn = model_classifier , verbose = 0)

grid_search_Keras_Class = GridSearchCV(model , param_grid , cv =10)

grid_search_Keras_Class.fit(xTraining, y_train)

print('Best parameters for efficiency classification {}'.format(grid_search_Keras_Class.best_params_))


Best parameters for efficiency classification {'batch_size': 10, 'epochs': 200}


In [43]:
print('Training Accuracy : ',accuracy_score(yTraining, grid_search_Keras_Class.predict(xTraining)))
print('Testing accuracy : ',accuracy_score(yTesting, grid_search_Keras_Class.predict(xTesting)))


The Train Accuracy score is 0.875
The Test Accuracy score is 0.890625
